In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from datasets import load_dataset
from dataclasses import dataclass
import math
import matplotlib.pyplot as plt
import tiktoken
import transformers

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
ds = load_dataset("KingNish/reasoning-base-20k")
ds = ds['train'].train_test_split(test_size=0.1)

README.md: 0.00B [00:00, ?B/s]

combined_reasoning.json:   0%|          | 0.00/307M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19944 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['user', 'reasoning', 'assistant', 'template', 'conversations'],
        num_rows: 17949
    })
    test: Dataset({
        features: ['user', 'reasoning', 'assistant', 'template', 'conversations'],
        num_rows: 1995
    })
})

In [5]:
model = tiktoken.encoding_for_model('gpt-2')
special_token_list = ['<|im_start|>', '<|im_end|>', "user", "reasoning", 'assistant', '<|PAD|>']
sp_tokens = {token : model.n_vocab+i for i, token in enumerate(special_token_list)}
sp_tokens.update(model._special_tokens)
model = tiktoken.Encoding(
    name="p50k_with_custom",
    pat_str=model._pat_str,
    mergeable_ranks=model._mergeable_ranks,
    special_tokens=sp_tokens
)
model.n_vocab

50263

In [6]:
model._special_tokens

{'<|im_start|>': 50257,
 '<|im_end|>': 50258,
 'user': 50259,
 'reasoning': 50260,
 'assistant': 50261,
 '<|PAD|>': 50262,
 '<|endoftext|>': 50256}

In [7]:
def encode(text, append_eot = False):
    tokens = model.encode(text, allowed_special = set(model._special_tokens.keys())) # forcefully allowing every special tokens
    if append_eot == True:
        tokens.append(50256)
    return tokens

def decode(tokens : list[int]):
    return model.decode(tokens)

In [8]:
@dataclass
class ModelArgs:
    vocab_size : int = model.n_vocab
    max_seq_len : int = 1280
    model_dim : int = 768
    padding_idx : int = 50262
    num_hidden_layers : int = 6
    intermediate_dim: int = 768
    n_kv_heads: int = 4
    n_head: int = 8
    rms_norm_eps : float = 1e-6
    bias : bool = False
    lr : float = 8e-4

In [9]:
class ReasoningDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, i):
        return self.data[i]['template']

    def __len__(self):
        return self.data.num_rows

In [10]:
def collate_fn(batch : list[str], max_seq_len : int = 1024, pad_id : int = 50262):
    batch_tokens = []
    for text in batch:
        tokens = encode(text, True)[:max_seq_len+1]
        token_len = len(tokens)
        pad_len = max(0, max_seq_len+1-token_len)
        if pad_len:
            tokens = tokens + [pad_id] * pad_len
        batch_tokens.append(tokens)
    return torch.tensor(batch_tokens, dtype=torch.long).to(device)
        

In [11]:
def calculate_mask(batch_x : torch.Tensor, pad_id : int = 50262):
    B, T = batch_x.shape
    causal_mask = torch.tril(torch.ones(1, T, T)).to(device)
    pad_mask = (batch_x!=pad_id).to(device)
    key_mask = pad_mask[:, None, :] # B, 1, T
    query_mask = pad_mask[:, :, None] # B, T, 1
    final_mask = causal_mask  * key_mask * query_mask
    return final_mask.to(device)

In [12]:
dataset = ReasoningDataset(ds['train'])
dataloader = DataLoader(dataset, batch_size = 8, collate_fn = lambda x: collate_fn(x, max_seq_len = 1280, pad_id = 50262), shuffle=True)

In [13]:
val_dataset = ReasoningDataset(ds['test'])
val_dataloader = DataLoader(val_dataset, batch_size = 8, collate_fn = lambda x: collate_fn(x, max_seq_len = 1280, pad_id = 50262), shuffle=True)

In [14]:
# for batch in dataloader:
#     break

In [15]:
# batch[3].tolist()[800:]

In [16]:
# batch.shape

In [17]:
# mask = calculate_mask(batch)

In [18]:
# mask.shape

In [19]:
# plt.imshow(mask[1].detach().cpu().numpy())

In [20]:
class RMSNorm(nn.Module):
    def __init__(self, dim : int, eps : float=1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(data=torch.ones(dim))
    def forward(self, x : torch.Tensor):
        variance = x.pow(2).mean(dim=-1, keepdims=True).type_as(x)
        return x * torch.rsqrt(variance+self.eps) * self.weight # rsqrt is same as 1/sqrt

In [21]:
class RoPE(nn.Module):
    def __init__(self, max_seq_len : int = 1024, d : int = 256, k : float = 10000.0, device : str = 'cpu'):
        super().__init__()
        self.d = d
        self.max_seq_len = max_seq_len
        self.device=device
        freqs, sin, cos = self.precompute_freqs(k=k)
        self.register_buffer('freqs', freqs.to(device))
        self.register_buffer('sin', sin.to(device))
        self.register_buffer('cos', cos.to(device))

    @torch.no_grad()
    def precompute_freqs(self, k : float = 10000.0):
        theta = 1/(k**(torch.arange(0, self.d, 2.0)/self.d))
        pos = torch.arange(self.max_seq_len).unsqueeze(1)
        freqs = pos*theta
        cos = torch.cos(freqs).to(self.device)
        sin = torch.sin(freqs).to(self.device)
        # print(theta.shape, pos.shape, freqs.shape)
        # print(theta, pos, freqs, sin, cos)
        return freqs, sin, cos
    def apply_rope(self, x : torch.Tensor):
        """Assumes x to be B, H, T,D"""
        B, H, T, D = x.shape
        x_reshaped = x.view(*x.shape[:-1], self.d//2, 2)
        x1 = x_reshaped[...,0]
        x2 = x_reshaped[...,1]

        cos = self.cos[:T, ...]
        sin = self.sin[:T, ...]
        stacked = torch.stack([x1 * cos - x2 * sin, 
                              x1 * sin + x2 * cos], dim=-1) # stack on last dimension
        out = stacked.view(x.shape)
        return out
    def forward(self, x : torch.Tensor):
        return self.apply_rope(x)

In [22]:
class LlamaMLP(nn.Module):
    def __init__(self, dim : int = 256, intermediate_dim : int = 256, bias : bool = True):
        super(LlamaMLP, self).__init__()
        self.d = dim
        self.intermediate_dim = intermediate_dim
        self.gate = nn.Linear(dim, intermediate_dim, bias=bias)
        self.up = nn.Linear(dim, intermediate_dim, bias=bias)
        self.down = nn.Linear(intermediate_dim, dim, bias=bias)
        self.activation_fn = F.silu

    def forward(self, x : torch.Tensor):
        # SwigLU(q, b) = SiLU(a) * b
        # final layer is W*(SwiGLU(x)) = W * (SiLU(x) * (W*x))
        return self.down(self.activation_fn(self.gate(x)) * self.up(x))

In [23]:
def repeat_kv(module : nn.Module, x : torch.Tensor, n_reps : int):
    B, H, T, D = x.shape
    if n_reps == 1:
        return x
    else:
        return x[:, :, None, :, :].expand(B, H, n_reps, T, D).reshape(B, H*n_reps, T, D)

In [24]:
class LlamaAttention(nn.Module):
    def __init__(self, dim : int = 256, n_kv_heads : int = 4, n_head : int = 8, max_seq_len : int = 1024):
        super(LlamaAttention, self).__init__()
        self.dim = dim
        self.n_kv_heads = n_kv_heads
        self.n_head = n_head
        self.head_dim = dim // n_head

        self.w_q = nn.Linear(dim, self.head_dim * self.n_head, bias=False)
        self.w_k = nn.Linear(dim, self.head_dim * self.n_kv_heads, bias=False)
        self.w_v = nn.Linear(dim, self.head_dim * self.n_kv_heads, bias=False)
        self.w_o = nn.Linear(self.head_dim * self.n_head, dim, bias=False)
        self.rotary_embedding = RoPE(max_seq_len = max_seq_len, d = self.head_dim)
    
    def forward(self, x : torch.Tensor, mask : torch.Tensor = None):
        """mask is filled with -inf at the position where the attn to be ignored
        x is of shape, B, T, D
        mask is of shape B, T, T"""
        B, T, D = x.shape
        
        # make all as shape B, H, T, head_dim
        Q = self.w_q(x).view(B, T, self.n_head, self.head_dim).transpose(1,2)
        K = self.w_k(x).view(B, T, self.n_kv_heads, self.head_dim).transpose(1,2) # on top of this repeatations are needed
        V = self.w_v(x).view(B, T, self.n_kv_heads, self.head_dim).transpose(1,2) # on top of this repeatations are needed

        Q = self.rotary_embedding(Q)
        K = self.rotary_embedding(K)
        
        n_reps = self.n_head // self.n_kv_heads
        K = repeat_kv(self, K, n_reps)
        V = repeat_kv(self, V, n_reps) 

        attn_scores = torch.matmul(Q, K.transpose(2,3)) / math.sqrt(self.head_dim) # B, H, T, D * B, H, D, T => B, H, T, T
        if mask is not None:
            mask = mask[:, :T, :T] # forcefully making of time steps equal to x
            attn_scores = attn_scores.masked_fill(mask.unsqueeze(1)==0, -1e9)
        
        attn_scores = F.softmax(attn_scores.float(), dim=-1)
        attn_output = torch.matmul(attn_scores, V) # B, H, T, T * B, H, T, D => B, H, T, D
        
        attn_output = attn_output.transpose(1,2)
        attn_output = attn_output.reshape(B, T, -1).contiguous() # reshape back to B, T, D from B, H, T, D
        
        attn_output = self.w_o(attn_output)
        return attn_output, attn_scores

In [25]:
# attn = LlamaAttention()

In [26]:
def count_params(module):
    total = 0
    for p in module.parameters():
        v = 1
        for d in p.shape:
            v *= d
        total += v
    return total

In [27]:
# x = torch.rand(32, 1024, 256)
# y, att_s = attn(x)

In [28]:
# y.shape, att_s.shape

In [29]:
class LlamaDecoder(nn.Module):
    def __init__(self, hidden_dim : int = 256, intermediate_dim : int = 256, n_kv_heads : int = 4, n_head : int = 8, max_seq_len : int = 1024):
        super(LlamaDecoder, self).__init__()
        self.rms_norm = RMSNorm(dim = hidden_dim) # eps needed
        self.self_attn = LlamaAttention(dim = hidden_dim, n_kv_heads = n_kv_heads, n_head = n_head, max_seq_len = max_seq_len)
        self.mlp = LlamaMLP(dim = hidden_dim, intermediate_dim = intermediate_dim) # bias needed
        self.hidden_dim = hidden_dim
        self.n_kv_heads = n_kv_heads
        self.n_head = n_head

    def forward(self, hidden_states : torch.Tensor, mask : torch.Tensor = None):
        """hidden_steps of shape B, T, D"""
        # print(torch.isnan(hidden_states).any())
        state = self.rms_norm(hidden_states)
        # print(torch.isnan(state).any())
        attn_output, attn_scores = self.self_attn(state, mask)
        # print(torch.isnan(attn_output).any())

        hidden_states = hidden_states + attn_output
        # print(torch.isnan(hidden_states).any())
        

        state = self.rms_norm(hidden_states)
        # print(torch.isnan(state).any())
        
        state = self.mlp(state)
        # print(torch.isnan(state).any())

        hidden_states = hidden_states + state
        # print(torch.isnan(hidden_states).any())
        # print('========================================')
        return hidden_states

In [30]:
# decoder = LlamaDecoder()
# count_params(decoder)

In [31]:
# y = decoder(x)
# y.shape

In [32]:
class Llama(nn.Module):
    def __init__(self, config : ModelArgs):
        super(Llama, self).__init__()
        self.config = config
        self.embedding_table = nn.Embedding(num_embeddings = config.vocab_size, embedding_dim = config.model_dim, padding_idx = config.padding_idx)
        self.decoder_layers = nn.ModuleList([
                                                LlamaDecoder(hidden_dim = config.model_dim, intermediate_dim = config.intermediate_dim,
                                                          n_kv_heads = config.n_kv_heads, n_head = config.n_head, max_seq_len = config.max_seq_len)
                                                for _ in range(config.num_hidden_layers)
                                            ])
        self.rms_norm = RMSNorm(dim = config.model_dim, eps = config.rms_norm_eps)
        self.mlp = LlamaMLP(dim = config.model_dim, intermediate_dim = config.intermediate_dim, bias = config.bias)
        self.proj_head = nn.Linear(config.model_dim, config.vocab_size, bias=False)

    def forward(self, x : torch.Tensor, mask : torch.Tensor = None):
        """X is in shape B, T
        mask is in shape B, T, T"""
        state = self.embedding_table(x)
        for dec_layer in self.decoder_layers:
            state = dec_layer(state, mask)
        
        state = self.rms_norm(state)
        state = self.mlp(state)
        state = self.proj_head(state)
        return state
    def generate(self, x : torch.Tensor, max_token : int = 1024):
        """x is of shape B, T"""
        for _ in range(max_token):
            x = x[:, -self.config.max_seq_len:]
            mask = calculate_mask(x, pad_id = 50262)
            state = self.embedding_table(x)
            for dec_layer in self.decoder_layers:
                state = dec_layer(state, mask)
            
            state = self.rms_norm(state)
            
            last_step_pred = state[:, -1, :] # B, 1, Vocab_size
            last_step_pred = self.mlp(last_step_pred)
            last_step_pred = self.proj_head(last_step_pred)
            probs = F.softmax(last_step_pred, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, idx_next), dim=1) # (B, T+1)
            yield idx_next

In [33]:
llama_model = Llama(config = ModelArgs).to(device)
print(f"Model size: {count_params(llama_model)/10**6}M parameters")

Model size: 100.226304M parameters


In [34]:
optimizer = torch.optim.AdamW(llama_model.parameters(), lr = ModelArgs.lr, betas = (0.9, 0.95), weight_decay = 0.2)

In [35]:
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps = 5625)

In [36]:
# for idx in llama_model.generate(torch.tensor([[50257]], dtype=torch.long, device=device), max_token=50):
#     print(decode(idx[0].tolist()), end='|')

In [37]:
def answer(question, max_token=1280, end = '|'):
    f_text = f"<|im_start|>user\n{question}<|im_end|>\n"
    tokens = encode(f_text)
    print(f_text)
    llama_model.eval()
    for idx in llama_model.generate(torch.tensor([tokens], dtype=torch.long, device=device), max_token=1280):
        print(decode(idx[0].tolist()), end=end)

In [38]:
# answer('Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1.')

In [39]:
# y = llama_model(batch[:, :-1], mask[:, :-1, :-1])

In [40]:
# y.shape

In [41]:
grad_accumulation_step = 16
for b in range(40):
    print(f"EPOCH: {b+1} / 40")
    llama_model.train()
    optimizer.zero_grad()
    for i, batch in enumerate(dataloader):
        x = batch[:, :-1]
        B, T = x.shape
        y = batch[:, 1:]
        mask = calculate_mask(x, ModelArgs.padding_idx)
        y_pred = llama_model(x, mask).view(B*T, ModelArgs.vocab_size)
        y = y.reshape(B*T,)
        loss = F.cross_entropy(y_pred, y)
        loss = loss / grad_accumulation_step
        loss.backward()
        if (i+1)%grad_accumulation_step==0:
            # print('Applying step')
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        if i%48==0:
            print(f"TRAIN Batch {i} => Loss: {round(loss.item()*grad_accumulation_step, 5)}")
    
    # apply for last batch
    if (i+1)%grad_accumulation_step!=0:
        # print('Applying step')
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    llama_model.eval()
    with torch.no_grad():
        for i, batch in enumerate(val_dataloader):
            x = batch[:, :-1]
            B, T = x.shape
            y = batch[:, 1:]
            mask = calculate_mask(x, ModelArgs.padding_idx)
            y_pred = llama_model(x, mask).view(B*T, ModelArgs.vocab_size)
            y = y.reshape(B*T,)
            loss = F.cross_entropy(y_pred, y)
            if i%50==0:
                print(f"EVAL Batch {i} => Loss: {round(loss.item(), 5)}")
    if b%5==0:
        # sample text generation test of reasoning
        answer('Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1.', end='')
        torch.save(llama_model.state_dict(), f'model_{b}.pth')

EPOCH: 1 / 40


TRAIN Batch 0 => Loss: 10.82988


TRAIN Batch 48 => Loss: 10.83492


TRAIN Batch 96 => Loss: 10.82574


TRAIN Batch 144 => Loss: 10.82212


...

<|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|>

<|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|>

EPOCH: 17 / 40


TRAIN Batch 0 => Loss: 0.57213


TRAIN Batch 48 => Loss: 0.52333


In [ ]:
answer('Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1.',
      end = "")

In [ ]:
x = torch.randint(0, 1280, (8, 1280))

In [ ]:
torch.save(llama_model.state_dict(), 'model.pth')

In [ ]:
state_dict = torch.load('model.pth', map_location=torch.device(device)) # Load to CPU

In [ ]:
torch.cuda.empty_cache()

In [ ]:
llama_model_ = Llama(config = ModelArgs) #.to(device)
llama_model_.load_state_dict(state_dict)

In [ ]:
a1 = llama_model(x.to('cuda'))
a2 = llama_model_(x.to('cpu'))

In [ ]:
torch.allclose(a1.cpu(), a2, atol=1e-5, rtol=1e-4)